import pandas as pd

# Read local file (keep read_csv as requested)
df_raw = pd.read_csv("worldbank_macro_usa.csv")

df_raw.head()


In [8]:
import pandas as pd

# Read local file (keep read_csv as requested)
df_raw = pd.read_csv("/Users/user/Downloads/P_Data_Extract_From_World_Development_Indicators (2)/33fc69b7-b3a0-45be-80a7-b2f173bf0b0c_Data.csv")
df_raw = df_raw.drop(labels=[3,4,5,6,7], axis=0)
df_raw.head()


,Series Name,Series Code,Country Name,Country Code,1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],...,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,United States,USA,8.500000,7.700000,7.100000,6.100000,5.800000,7.100000,...,5.280000,4.869000,4.355000,3.896000,3.669000,8.055000,5.349000,3.650000,3.638000,4.022000
1,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,United States,USA,9.143147,5.744813,6.501684,7.630964,11.254471,13.549202,...,0.118627,1.261583,2.130110,2.442583,1.812210,1.233584,4.697859,8.002800,4.116338,2.949525
2,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,United States,USA,-0.205619,5.388034,4.624187,5.535206,3.165988,-0.256776,...,2.945550,1.819451,2.457622,2.966505,2.583825,-2.163029,6.055053,2.512375,2.887556,2.793001


In [9]:
# Keep only the USA rows (safety check)
df_raw = df_raw[df_raw["Country Code"] == "USA"]

# Identify year columns automatically
year_cols = [c for c in df_raw.columns if "[YR" in c]

df_raw = df_raw[["Series Name"] + year_cols]


In [ ]:
# Convert from wide to long format
df_long = df_raw.melt(
    id_vars="Series Name",
    value_vars=year_cols,
    var_name="year",
    value_name="value"
)

# Extract numeric year
df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.reset_index()


In [21]:
# Pivot so that each variable is a column
df_ts = df_long.pivot(
    index="year",
    columns="Series Name",
    values="value"
)

df_ts = df_ts.sort_index()
df_ts.columns.name = None

# df_ts = df_ts.rename(columns={"year": "Year"})
df_ts


,GDP growth (annual %),"Inflation, consumer prices (annual %)","Unemployment, total (% of total labor force) (national estimate)"
year,,,
1975,-0.205619,9.143147,8.500
1976,5.388034,5.744813,7.700
1977,4.624187,6.501684,7.100
1978,5.535206,7.630964,6.100
1979,3.165988,11.254471,5.800
1980,-0.256776,13.549202,7.100
1981,2.537701,10.334715,7.600
1982,-1.803014,6.131427,9.700
1983,4.583791,3.212435,9.600


In [22]:
df_ts = df_ts.rename(columns={
    "GDP growth (annual %)": "gdp_growth",
    "Inflation, consumer prices (annual %)": "inflation",
    "Unemployment, total (% of total labor force) (national estimate)": "unemployment"
})

# Convert to numeric
df_ts = df_ts.apply(pd.to_numeric, errors="coerce")

df_ts = df_ts.dropna()
df_ts


,gdp_growth,inflation,unemployment
year,,,
1975,-0.205619,9.143147,8.500
1976,5.388034,5.744813,7.700
1977,4.624187,6.501684,7.100
1978,5.535206,7.630964,6.100
1979,3.165988,11.254471,5.800
1980,-0.256776,13.549202,7.100
1981,2.537701,10.334715,7.600
1982,-1.803014,6.131427,9.700
1983,4.583791,3.212435,9.600
